In [1]:
import os  
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
os.makedirs('newdat', exist_ok=True)  
games = pd.read_csv('newdat/games_v1.csv')

In [3]:
games.head()

,GAME_DATE_EST,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,...,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,NAME_HOME,COACH_HOME,NAME_AWAY,COACH_AWAY
0,3/12/2022,22101005,2021,1610612748,104.0,0.398,0.760,0.333,23.0,53.0,...,0.422,0.875,0.357,21.0,46.0,0,Heat,Erik Spoelstra,Timberwolves,Ryan Saunders
1,3/12/2022,22101006,2021,1610612741,101.0,0.443,0.933,0.429,20.0,46.0,...,0.419,0.824,0.208,19.0,40.0,1,Bulls,Jim Boylen,Cavaliers,John Beilein
2,3/12/2022,22101007,2021,1610612759,108.0,0.412,0.813,0.324,28.0,52.0,...,0.489,1.000,0.389,23.0,47.0,0,Spurs,Gregg Popovich,Pacers,Nate McMillan
3,3/12/2022,22101008,2021,1610612744,122.0,0.484,0.933,0.400,33.0,55.0,...,0.413,0.696,0.386,27.0,39.0,1,Warriors,Steve Kerr,Bucks,Mike Budenholzer
4,3/12/2022,22101009,2021,1610612743,115.0,0.551,0.750,0.407,32.0,39.0,...,0.471,0.760,0.387,28.0,50.0,0,Nuggets,Michael Malone,Raptors,Nick Nurse


In [17]:
season = 2020
for season in range(2003,2022,1):
    games_s = games[games['SEASON'] == season]
    ## use a dictionary to count the round number
    round_dict = {}
    for ele in set(games_s['NAME_HOME']):
        round_dict[ele] = 0
    games_s['round_number_home'] = 0
    games_s['round_number_away'] = 0
    ## add round numbers for each team to the dataset.

    for row in games_s[::-1].itertuples():
        home_team = row.NAME_HOME
        games_s.at[row.Index, 'round_number_home'] = round_dict[home_team] + 1
        round_dict[home_team] += 1

        away_team = row.NAME_AWAY
        games_s.at[row.Index, 'round_number_away'] = round_dict[away_team] + 1
        round_dict[away_team] += 1
    # PTS_home_cur_season, FG_PCT_home_cur_season, FT_PCT_home_cur_season, 
    # FG3_PCT_home_cur_season, AST_home_cur_season, REB_home_cur_season 
    games_s['PTS_home_cur_season'] = 0.0
    games_s['FG_PCT_home_cur_season'] = 0.0
    games_s['FT_PCT_home_cur_season'] = 0.0
    games_s['FG3_PCT_home_cur_season'] = 0.0
    games_s['AST_home_cur_season'] = 0.0
    games_s['REB_home_cur_season'] = 0.0
    games_s['WINRATE_home_cur_season'] = 0.0
    games_s['PTS_LOST_home_cur_season'] = 0.0

    games_s['PTS_away_cur_season'] = 0.0
    games_s['FG_PCT_away_cur_season'] = 0.0
    games_s['FT_PCT_away_cur_season'] = 0.0
    games_s['FG3_PCT_away_cur_season'] = 0.0
    games_s['AST_away_cur_season'] = 0.0
    games_s['REB_away_cur_season'] = 0.0
    games_s['WINRATE_away_cur_season'] = 0.0
    games_s['PTS_LOST_away_cur_season'] = 0.0

    for row in games_s[::-1].itertuples():
        round_home = row.round_number_home ## round number for home team
        round_away = row.round_number_away ## round number for away team
        if round_home == 1 and round_away == 1:
             continue
        else:
            ## pts for home team
            if round_home > 1:
                last_match_home = games_s[ ((games_s['round_number_home'] == round_home - 1)  & (games_s['NAME_HOME'] == row.NAME_HOME)) | 
                                           ((games_s['round_number_away'] == round_home - 1)  & (games_s['NAME_AWAY'] == row.NAME_HOME))]
                ## if name_home on this row is the target team, we use home stats for calculating the features
                if last_match_home['NAME_HOME'].values[0] == row.NAME_HOME: 
                    ### avg pts for previous games
                    games_s.at[row.Index, 'PTS_home_cur_season'] = (last_match_home['PTS_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['PTS_home'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'PTS_LOST_home_cur_season'] = (last_match_home['PTS_LOST_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['PTS_away'].values[0]) / (round_home - 1)
                    ### avg fg_pct for previous games
                    games_s.at[row.Index, 'FG_PCT_home_cur_season'] = (last_match_home['FG_PCT_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['FG_PCT_home'].values[0]) / (round_home - 1)
                    ### avg ft_pct for ptrvious games
                    games_s.at[row.Index, 'FT_PCT_home_cur_season'] = (last_match_home['FT_PCT_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['FT_PCT_home'].values[0]) / (round_home - 1)
                    ### avg FG3_PCT_home_cur_season
                    games_s.at[row.Index, 'FG3_PCT_home_cur_season'] = (last_match_home['FG3_PCT_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['FG3_PCT_home'].values[0]) / (round_home - 1)
                    ### avg AST_home_cur_season
                    games_s.at[row.Index, 'AST_home_cur_season'] = (last_match_home['AST_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['AST_home'].values[0]) / (round_home - 1)
                    ### avg REB_home_cur_season 
                    games_s.at[row.Index, 'REB_home_cur_season'] = (last_match_home['REB_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['REB_home'].values[0]) / (round_home - 1)

                    games_s.at[row.Index, 'WINRATE_home_cur_season'] = (last_match_home['WINRATE_home_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['HOME_TEAM_WINS'].values[0]) / (round_home - 1)

                else: ## else, we use away team stats for calculation
                    games_s.at[row.Index, 'PTS_home_cur_season'] = (last_match_home['PTS_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['PTS_away'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'PTS_LOST_home_cur_season'] = (last_match_home['PTS_LOST_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['PTS_home'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'FG_PCT_home_cur_season'] = (last_match_home['FG_PCT_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['FG_PCT_away'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'FT_PCT_home_cur_season'] = (last_match_home['FT_PCT_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['FT_PCT_away'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'FG3_PCT_home_cur_season'] = (last_match_home['FG3_PCT_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['FG3_PCT_away'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'AST_home_cur_season'] = (last_match_home['AST_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['AST_away'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'REB_home_cur_season'] = (last_match_home['REB_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + last_match_home['REB_away'].values[0]) / (round_home - 1)
                    games_s.at[row.Index, 'WINRATE_home_cur_season'] = (last_match_home['WINRATE_away_cur_season'].values[0] * (round_home - 2) 
                                                                  + (1-last_match_home['HOME_TEAM_WINS'].values[0])) / (round_home - 1)

            ## for away team
            if round_away == 1: continue
            last_match_away = games_s[ ((games_s['round_number_home'] == round_away - 1)  & (games_s['NAME_HOME'] == row.NAME_AWAY)) | 
                                       ((games_s['round_number_away'] == round_away - 1)  & (games_s['NAME_AWAY'] == row.NAME_AWAY))]
            if last_match_away['NAME_HOME'].values[0] == row.NAME_AWAY:
                ### avg pts for previous games
                games_s.at[row.Index, 'PTS_away_cur_season'] = (last_match_away['PTS_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['PTS_home'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'PTS_LOST_away_cur_season'] = (last_match_away['PTS_LOST_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['PTS_away'].values[0]) / (round_away - 1)
                ### avg fg_pct for previous games
                games_s.at[row.Index, 'FG_PCT_away_cur_season'] = (last_match_away['FG_PCT_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['FG_PCT_home'].values[0]) / (round_away - 1)
                ### avg ft_pct for ptrvious games
                games_s.at[row.Index, 'FT_PCT_away_cur_season'] = (last_match_away['FT_PCT_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['FT_PCT_home'].values[0]) / (round_away - 1)
                ### avg FG3_PCT_home_cur_season
                games_s.at[row.Index, 'FG3_PCT_away_cur_season'] = (last_match_away['FG3_PCT_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['FG3_PCT_home'].values[0]) / (round_away - 1)
                ### avg AST_home_cur_season
                games_s.at[row.Index, 'AST_away_cur_season'] = (last_match_away['AST_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['AST_home'].values[0]) / (round_away - 1)
                ### avg REB_home_cur_season 
                games_s.at[row.Index, 'REB_away_cur_season'] = (last_match_away['REB_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['REB_home'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'WINRATE_away_cur_season'] = (last_match_away['WINRATE_home_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['HOME_TEAM_WINS'].values[0]) / (round_away - 1)
            else: 
                games_s.at[row.Index, 'PTS_away_cur_season'] = (last_match_away['PTS_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['PTS_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'PTS_LOST_away_cur_season'] = (last_match_away['PTS_LOST_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['PTS_home'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'PTS_away_cur_season'] = (last_match_away['PTS_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['PTS_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'FG_PCT_away_cur_season'] = (last_match_away['FG_PCT_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['FG_PCT_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'FT_PCT_away_cur_season'] = (last_match_away['FT_PCT_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['FT_PCT_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'FG3_PCT_away_cur_season'] = (last_match_away['FG3_PCT_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['FG3_PCT_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'AST_away_cur_season'] = (last_match_away['AST_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['AST_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'REB_away_cur_season'] = (last_match_away['REB_away_cur_season'].values[0] * (round_away - 2) 
                                                              + last_match_away['REB_away'].values[0]) / (round_away - 1)
                games_s.at[row.Index, 'WINRATE_away_cur_season'] = (last_match_away['WINRATE_away_cur_season'].values[0] * (round_away - 2) 
                                                              + (1-last_match_away['HOME_TEAM_WINS'].values[0])) / (round_away - 1)



    games_s.to_csv('newdat/games'+str(season)+'_v1.csv',index=False)  
    print("season", season, "done")

season 2003 done
season 2004 done
season 2005 done
season 2006 done
season 2007 done
season 2008 done
season 2009 done
season 2010 done
season 2011 done
season 2012 done
season 2013 done
season 2014 done
season 2015 done
season 2016 done
season 2017 done
season 2018 done
season 2019 done
season 2020 done
season 2021 done


In [18]:
games_s

,GAME_DATE_EST,GAME_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,...,WINRATE_home_cur_season,PTS_LOST_home_cur_season,PTS_away_cur_season,FG_PCT_away_cur_season,FT_PCT_away_cur_season,FG3_PCT_away_cur_season,AST_away_cur_season,REB_away_cur_season,WINRATE_away_cur_season,PTS_LOST_away_cur_season
0,3/12/2022,22101005,2021,1610612748,104.0,0.398,0.760,0.333,23.0,53.0,...,0.675676,104.878378,114.625000,0.453722,0.773792,0.350569,25.652778,44.763889,0.569444,111.527778
1,3/12/2022,22101006,2021,1610612741,101.0,0.443,0.933,0.429,20.0,46.0,...,0.628571,110.157143,106.492958,0.466085,0.760732,0.348465,25.084507,44.577465,0.563380,103.915493
2,3/12/2022,22101007,2021,1610612759,108.0,0.412,0.813,0.324,28.0,52.0,...,0.402778,112.055556,109.929577,0.459324,0.757324,0.337549,24.788732,44.873239,0.338028,112.661972
3,3/12/2022,22101008,2021,1610612744,122.0,0.484,0.933,0.400,33.0,55.0,...,0.694444,104.972222,114.222222,0.465139,0.771111,0.367389,23.541667,46.819444,0.597222,110.527778
4,3/12/2022,22101009,2021,1610612743,115.0,0.551,0.750,0.407,32.0,39.0,...,0.569444,108.347222,108.887324,0.443845,0.754423,0.354887,22.056338,45.042254,0.549296,107.408451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,10/4/2021,12100007,2021,1610612759,111.0,0.465,0.760,0.353,22.0,60.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1072,10/4/2021,12100008,2021,1610612757,107.0,0.446,0.773,0.421,21.0,53.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1073,10/4/2021,12100009,2021,1610612758,117.0,0.473,0.684,0.340,22.0,51.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1074,10/4/2021,12100010,2021,1610612746,103.0,0.432,0.760,0.333,26.0,52.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
games2004 = pd.read_csv('newdat/games2004_v1.csv')
games2005 = pd.read_csv('newdat/games2005_v1.csv')
games2006 = pd.read_csv('newdat/games2006_v1.csv')
games2007 = pd.read_csv('newdat/games2007_v1.csv')
games2008 = pd.read_csv('newdat/games2008_v1.csv')
games2009 = pd.read_csv('newdat/games2009_v1.csv')
games2010 = pd.read_csv('newdat/games2010_v1.csv')
games2011 = pd.read_csv('newdat/games2011_v1.csv')
games2012 = pd.read_csv('newdat/games2012_v1.csv')
games2013 = pd.read_csv('newdat/games2013_v1.csv')
games2014 = pd.read_csv('newdat/games2014_v1.csv')
games2015 = pd.read_csv('newdat/games2015_v1.csv')
games2016 = pd.read_csv('newdat/games2016_v1.csv')
games2017 = pd.read_csv('newdat/games2017_v1.csv')
games2018 = pd.read_csv('newdat/games2018_v1.csv')
games2019 = pd.read_csv('newdat/games2019_v1.csv')
games2020 = pd.read_csv('newdat/games2020_v1.csv')
games2021 = pd.read_csv('newdat/games2021_v1.csv')

In [20]:
games_extended = pd.concat([games2021, games2020, games2019, games2018,games2017,games2016,games2015,games2014,games2013,games2012,games2011,games2010,
           games2009,games2008,games2007,games2006,games2005,games2004],ignore_index = True)

In [27]:
games_extended.to_csv('newdat/games_extended.csv',index=False) 

In [33]:
for season in range(2003,2022,1):
    print(games[games['SEASON'] == season]['GAME_DATE_EST'].iloc[-1])

10/5/2003
10/22/2004
10/10/2005
10/9/2006
10/6/2007
10/5/2008
10/1/2009
10/4/2010
12/16/2011
10/6/2012
10/5/2013
10/4/2014
10/2/2015
10/1/2016
9/30/2017
9/28/2018
10/4/2019
12/11/2020
10/3/2021
